# Notebook Intentions

The purpose of this notebook is build the methodology for extracting events from specific event files. We will use the FullYearConsolidated18 file as the "ground truth" since it contains aggregate counts of event and event sub types. 

## Structure

First we build a lookup dictionary for all respondents derived from the FullYearConsolidated18 table. This dictionary will contain the following metrics:

    - Total Office Based Events
    - Total Physician Office Based Events
    - Total Outpatient Events
    - Total Physician Outpatient Events
    - Total Emergency Room Events
    - Total Hospital Discharges
    - Total Nights Spent in Hospitals
    - Total Dental Care Events
    - Total Home Health Days
    - Total Agency Home Health Days
    - Total Non-Agency Home Health Days
    - Total Informal Home Health Days
    - Total Presciption Drugs + Refills
    
Then for each event data file, OfficeBasedVisits18, OutpatientVisits18, EmergencyRoomVisits18, HospitalInpatientStays18, DentalVisits18, HomeHealth18, and PrescribedMedicines18. We will identify the correct methodology for extracting event, such that the aggregate of events matches with the values found in the lookup dictionary.

## Conclusions

A sound methodology was identified for all events, we were able to generate a perfect match for all respondents for all event types. The only exception was home health events, however the inconsistencies (<5%) match up perfectly with reported missing data. Now that we can correctly extract events from event files for 2018, we will attempt to build a generalizable process that can be applied for all years. The end goal of this is to be able to generate a perfect "timeline" for each respondent, such that events are ordered throughout a year.

In [1]:
import os
from os.path import expanduser

import sys
sys.path.append(os.path.join(expanduser("~"), "meps", "meps_dev"))
# points to the project setting file

os.environ.setdefault("DJANGO_SETTINGS_MODULE", "meps_db.settings")
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

import django
django.setup();

# Respondents
from meps_db.components.models.full_year_consolidated_models import FullYearConsolidated18

# Events
from meps_db.components.models.office_based_visits_models import OfficeBasedVisits18
from meps_db.components.models.outpatient_visits_models import OutpatientVisits18
from meps_db.components.models.emergency_room_visits_models import EmergencyRoomVisits18
from meps_db.components.models.hospital_inpatient_stays_models import HospitalInpatientStays18
from meps_db.components.models.dental_visits_models import DentalVisits18
from meps_db.components.models.home_health_models import HomeHealth18
from meps_db.components.models.prescribed_medicines_models import PrescribedMedicines18

from datetime import date

## Helpers


In [2]:
def map_common_int(val):
    """ Takes a string value, returns an integer version or None if the value is in one of the following:
    -8 DK, -7 REFUSED, -1 INAPPLICABLE """
    
    if val in {"-15", "-8", "-7", "-1"}:
        return None
    else:
        return int(val)
    
def report_pct_of_pop(sub_pop, full_pop, desc):
    """ Takes a subpopulation size, a full population size and a description. Prints a statement description the
    size of the subpopulation with respect to the full population """
    
    print(f"{sub_pop:,} / {full_pop:,} ({sub_pop/full_pop*100:.2f}%) {desc}")

### Identify Respondents

Fetch all respondents from the Full Year Consolidated File, gather the following fields:

    - DUPERSID: UNIQUE PERSON ID
    - OBTOTV18: # OFFICE-BASED PROVIDER VISITS 2018
    - OBDRV18: # OFFICE-BASED PHYSICIAN VISITS 2018
    - OPTOTV18: # OUTPATIENT DEPT PROVIDER VISITS 2018 
    - OPDRV18 # OUTPATIENT DEPT PHYSICIAN VISITS 2018 
    - ERTOT18: # EMERGENCY ROOM VISITS 2018 
    - IPDIS18: # HOSPITAL DISCHARGES 2018
    - IPNGTD18 # NIGHTS IN HOSP FOR DISCHARGES, 2018 
    - DVTOT18 # DENTAL CARE VISITS 2018 
    - HHTOTD18: # HOME HEALTH PROVIDER DAYS 2018
    - HHAGD18 # AGENCY HOME HEALTH PROVIDER DAYS 2018
    - HHINDD18 # NON-AGENCY HOME HLTH PROVIDR DAYS 2018
    - HHINFD18 # INFORMAL HOME HLTH PROVIDER DAYS 2018 
    - RXTOT18: # PRESC MEDS INCL REFILLS 18
    
This data will be used to identify all respondents in other models and validate timelines are being generated correctly. 

OBDRV18 is a subset of OBTOTV18, and OPDRV18 is a subset of OPTOTV18. We will attempt to generate these subsets from their associated models.

In [3]:
# Identify Respondents

FYC_VARS = [
    "DUPERSID", "OBTOTV18", "OBDRV18", "OPTOTV18", "OPDRV18", "ERTOT18", "IPDIS18", "IPNGTD18",
    "DVTOT18", "HHTOTD18", "HHAGD18", "HHINDD18", "HHINFD18", "RXTOT18"
]

fyc_lookup = list(FullYearConsolidated18.objects.all().values(*FYC_VARS))
# convert to lookup dictionary 
fyc_lookup = {
    resp["DUPERSID"]: {
        "office_based_events_count": map_common_int(resp["OBTOTV18"]),
        "office_based_physician_events_count": map_common_int(resp["OBDRV18"]),
        "outpatient_events_count": map_common_int(resp["OPTOTV18"]),
        "outpatient_events_physican_count": map_common_int(resp["OPDRV18"]),
        "emergency_room_visit_count": map_common_int(resp["ERTOT18"]),
        "hospital_discharges_count": map_common_int(resp["IPDIS18"]),
        "nights_in_hospital_total": map_common_int(resp["IPNGTD18"]),
        "dental_care_visits_count": map_common_int(resp["DVTOT18"]),
        "home_health_provider_days_total": map_common_int(resp["HHTOTD18"]),
        "home_health_agency_days_total": map_common_int(resp["HHAGD18"]),
        "home_health_non_agency_days_total": map_common_int(resp["HHINDD18"]),
        "home_health_informal_days_total": map_common_int(resp["HHINFD18"]),
        "prescription_drugs_total": map_common_int(resp["RXTOT18"]),
    } 
    for resp in fyc_lookup
}

assert len(fyc_lookup) == len(set(fyc_lookup))

print(f"{len(fyc_lookup):,} Respondents")

30,461 Respondents


In [4]:
# example
fyc_lookup[list(fyc_lookup.keys())[0]]

{'office_based_events_count': 5,
 'office_based_physician_events_count': 5,
 'outpatient_events_count': 0,
 'outpatient_events_physican_count': 0,
 'emergency_room_visit_count': 0,
 'hospital_discharges_count': 0,
 'nights_in_hospital_total': 0,
 'dental_care_visits_count': 2,
 'home_health_provider_days_total': 0,
 'home_health_agency_days_total': 0,
 'home_health_non_agency_days_total': 0,
 'home_health_informal_days_total': 0,
 'prescription_drugs_total': 27}

## Office Based Visits

Fetch all office based visits. The following fields are gathered:

    - DUPERSID: UNIQUE PERSON ID
    - EVNTIDX: EVENT ID
    - OBDATEYR: EVENT DATE - YEAR
    - OBDATEMM: EVENT DATE - MONTH
    - SEEDOC_M18: DID P TALK TO MD THIS VISIT

    
First we will group office based visits with their associated DUPERSID. Then we will verify that the aggregate counts match those from the FYC data. Then we will attempt to recreate the # OFFICE-BASED PHYSICIAN VISITS 2018 field. SEEDOC_M18 appear to be the flag used to classify physican events.

In [5]:
OB_VARS = [
    "DUPERSID",  "EVNTIDX", "OBDATEYR", "OBDATEMM", "SEEDOC_M18",
]

office_based = list(OfficeBasedVisits18.objects.filter(DUPERSID__in=fyc_lookup.keys()).values(*OB_VARS))
respondents_ob = {}
for event in office_based:
    if event["DUPERSID"] not in respondents_ob:
        respondents_ob[event["DUPERSID"]] = []
    
    # generate date if possible
    if event["OBDATEMM"] != "-8":
        event_date = date(year=int(event["OBDATEYR"]), month=int(event["OBDATEMM"]), day=1)
    else:
        event_date = None
    
    # classify as physican visit
    physician_event = event["SEEDOC_M18"] in {"01"}
    
    respondents_ob[event["DUPERSID"]].append(
        {
            "event_id": event["EVNTIDX"],
            "date": event_date,
            "physician_event": physician_event
        }
    )

In [6]:
# Overview
total_respondents = len(fyc_lookup)
total_office_based_respondents = len(respondents_ob)
report_pct_of_pop(
    sub_pop=total_office_based_respondents, 
    full_pop=total_respondents, 
    desc="respondents have at least 1 Office Based Visit"
)

# validate event counts match
ob_match = 0
for resp, event_list in respondents_ob.items():
    total_ob_events = len(event_list)
    if fyc_lookup[resp]["office_based_events_count"] == total_ob_events:
        ob_match += 1

report_pct_of_pop(
    sub_pop=ob_match, 
    full_pop=total_office_based_respondents, 
    desc="respondents have consistent office based provider visit counts with the FYC data"
)

# validate physician event counts match
ob_phys_match = 0
for resp, event_list in respondents_ob.items():
    total_ob_phys_events = len([event for event in event_list if event["physician_event"]])
    if fyc_lookup[resp]["office_based_physician_events_count"] == total_ob_phys_events:
        ob_phys_match += 1

report_pct_of_pop(
    sub_pop=ob_phys_match, 
    full_pop=total_office_based_respondents, 
    desc="respondents have consistent office based physician visit counts with the FYC data"
)


22,519 / 30,461 (73.93%) respondents have at least 1 Office Based Visit
22,519 / 22,519 (100.00%) respondents have consistent office based provider visit counts with the FYC data
22,519 / 22,519 (100.00%) respondents have consistent office based physician visit counts with the FYC data


## Outpatient Visits

Fetch all outpatient based visits. The following fields are gathered:

    - DUPERSID: UNIQUE PERSON ID
    - EVNTIDX: EVENT ID
    - OPDATEYR: EVENT DATE - YEAR
    - OPDATEMM: EVENT DATE - MONTH
    - SEEDOC_M18: DID P TALK TO MD THIS VISIT

    
First we will group outpatien visits with their associated DUPERSID. Then we will verify that the aggregate counts match those from the FYC data. Then we will attempt to recreate the # OUTPATIENT DEPT PROVIDER VISITS 2018. SEEDOC_M18 appear to be the flag used to classify physican events.

In [7]:
OP_VARS = [
    "DUPERSID",  "EVNTIDX", "OPDATEYR", "OPDATEMM", "SEEDOC_M18"
]

outpatient_based = list(OutpatientVisits18.objects.filter(DUPERSID__in=fyc_lookup.keys()).values(*OP_VARS))
respondents_op = {}
for event in outpatient_based:
    if event["DUPERSID"] not in respondents_op:
        respondents_op[event["DUPERSID"]] = []
    
    # generate date if possible
    if event["OPDATEMM"] != "-8":
        event_date = date(year=int(event["OPDATEYR"]), month=int(event["OPDATEMM"]), day=1)
    else:
        event_date = None
    
    # classify as physican visit
    physician_event = event["SEEDOC_M18"] in {"01"} # event["DRSPLTY_M18"] in {"06", "08"}
    
    respondents_op[event["DUPERSID"]].append(
        {
            "event_id": event["EVNTIDX"],
            "date": event_date,
            "physician_event": physician_event
        }
    )

In [8]:
# Overview
total_respondents = len(fyc_lookup)
total_outpatient_respondents = len(respondents_op)
report_pct_of_pop(
    sub_pop=total_outpatient_respondents, 
    full_pop=total_respondents, 
    desc="respondents have at least 1 Outpatient Visit"
)

# validate event counts match
op_match = 0
for resp, event_list in respondents_op.items():
    total_op_events = len(event_list)
    if fyc_lookup[resp]["outpatient_events_count"] == total_op_events:
        op_match += 1

report_pct_of_pop(
    sub_pop=op_match, 
    full_pop=total_outpatient_respondents, 
    desc="respondents have consistent outpatient provider visit counts with the FYC data"
)

# validate physician event counts match
op_phys_match = 0
for resp, event_list in respondents_op.items():
    total_op_phys_events = len([event for event in event_list if event["physician_event"]])
    if fyc_lookup[resp]["outpatient_events_physican_count"] == total_op_phys_events:
        op_phys_match += 1

report_pct_of_pop(
    sub_pop=op_phys_match, 
    full_pop=total_outpatient_respondents, 
    desc="respondents have consistent outpatient physician visit counts with the FYC data"
)


5,455 / 30,461 (17.91%) respondents have at least 1 Outpatient Visit
5,455 / 5,455 (100.00%) respondents have consistent outpatient provider visit counts with the FYC data
5,455 / 5,455 (100.00%) respondents have consistent outpatient physician visit counts with the FYC data


## ER Visits

Fetch all Emergency Room visits. The following fields are gathered:

    - DUPERSID: UNIQUE PERSON ID
    - EVNTIDX: EVENT ID
    - ERDATEYR: EVENT DATE - YEAR
    - ERDATEMM: EVENT DATE - MONTH

    
First we will group Emergency Room visits with their associated DUPERSID. Then we will verify that the aggregate counts match those from the FYC data. 

In [9]:
ER_VARS = [
     "DUPERSID",  "EVNTIDX", "ERDATEYR", "ERDATEMM", 
]

emergency_room = list(EmergencyRoomVisits18.objects.filter(DUPERSID__in=fyc_lookup.keys()).values(*ER_VARS))
respondents_er = {}
for event in emergency_room:
    if event["DUPERSID"] not in respondents_er:
        respondents_er[event["DUPERSID"]] = []
     # generate date if possible
    if event["ERDATEMM"] != "-8":
        event_date = date(year=int(event["ERDATEYR"]), month=int(event["ERDATEMM"]), day=1)
    else:
        event_date = None
    
    respondents_er[event["DUPERSID"]].append(
        {
            "event_id": event["EVNTIDX"],
            "date": event_date,
        }
    )

In [10]:
# Overview
total_respondents = len(fyc_lookup)
total_er_respondents = len(respondents_er)
report_pct_of_pop(
    sub_pop=total_er_respondents, 
    full_pop=total_respondents, 
    desc="respondents have at least 1 Emergency Room Event"
)

# validate event counts match
er_match = 0
for resp, event_list in respondents_er.items():
    total_er_events = len(event_list)
    if fyc_lookup[resp]["emergency_room_visit_count"] == total_er_events:
        er_match += 1

report_pct_of_pop(
    sub_pop=er_match, 
    full_pop=total_er_respondents, 
    desc="respondents have consistent Emergency Room Event counts with the FYC data"
)

4,665 / 30,461 (15.31%) respondents have at least 1 Emergency Room Event
4,665 / 4,665 (100.00%) respondents have consistent Emergency Room Event counts with the FYC data


## Hospital Inpatient Stays

Fetch all Hospital Inpatient events. The following fields are gathered:

    - DUPERSID: UNIQUE PERSON ID
    - EVNTIDX: EVENT ID
    - IPENDYR: EVENT DATE - YEAR
    - IPENDMM: EVENT DATE - MONTH
    - NUMNIGHX: # OF NIGHTS IN HOSPITAL - EDITED/IMPUTED

    
First we will group hospital inpatients stays with their associated DUPERSID. Then we will verify that the aggregate counts match those from the FYC data, and that the total night's spent in hospitals match the FYC counts.

In [11]:
ER_VARS = ["DUPERSID", "EVNTIDX", "IPENDYR", "IPENDMM", "NUMNIGHX"]

hospital_inpatient_stays = list(
    HospitalInpatientStays18.objects.filter(DUPERSID__in=fyc_lookup.keys()).values(*ER_VARS)
)

respondents_his = {}
for event in hospital_inpatient_stays:
    if event["DUPERSID"] not in respondents_his:
        respondents_his[event["DUPERSID"]] = []
     # generate date if possible
    if event["IPENDMM"] != "-8":
        event_date = date(year=int(event["IPENDYR"]), month=int(event["IPENDMM"]), day=1)
    else:
        event_date = None
    
    respondents_his[event["DUPERSID"]].append(
        {
            "event_id": event["EVNTIDX"],
            "date": event_date,
            "nights": int(event["NUMNIGHX"])
        }
    )

In [12]:
# Overview
total_respondents = len(fyc_lookup)
total_his_respondents = len(respondents_his)
report_pct_of_pop(
    sub_pop=total_his_respondents, 
    full_pop=total_respondents, 
    desc="respondents have at least 1 Hospital Inpatient Stay"
)

# validate event counts match
his_match = 0
for resp, event_list in respondents_his.items():
    total_his = len(event_list)
    if fyc_lookup[resp]["hospital_discharges_count"] == total_his:
        his_match += 1

report_pct_of_pop(
    sub_pop=his_match, 
    full_pop=total_his_respondents, 
    desc="respondents have consistent Hospital Inpatient Stay counts with the FYC data"
)

# validate total night sums match
his_nights_match = 0
for resp, event_list in respondents_his.items():
    total_his_nights = sum([event["nights"] for event in event_list])
    if fyc_lookup[resp]["nights_in_hospital_total"] == total_his_nights:
        his_nights_match += 1

report_pct_of_pop(
    sub_pop=his_nights_match, 
    full_pop=total_his_respondents, 
    desc="respondents have consistent Sums of Nights spent in Hospital with the FYC data"
)


2,168 / 30,461 (7.12%) respondents have at least 1 Hospital Inpatient Stay
2,168 / 2,168 (100.00%) respondents have consistent Hospital Inpatient Stay counts with the FYC data
2,168 / 2,168 (100.00%) respondents have consistent Sums of Nights spent in Hospital with the FYC data


## Dental Visits

Fetch all Dental Visits. The following fields are gathered:

    - DUPERSID: UNIQUE PERSON ID
    - EVNTIDX: EVENT ID
    - DVDATEYR: EVENT DATE - YEAR
    - DVDATEMM: EVENT DATE - MONTH

    
First we will group dental visits with their associated DUPERSID. Then we will verify that the aggregate counts match those from the FYC data.

In [13]:
DV_VARS = ["DUPERSID", "EVNTIDX", "DVDATEYR", "DVDATEMM"]

dental_visits = list(
    DentalVisits18.objects.filter(DUPERSID__in=fyc_lookup.keys()).values(*DV_VARS)
)

respondents_dental_visits = {}
for event in dental_visits:
    if event["DUPERSID"] not in respondents_dental_visits:
        respondents_dental_visits[event["DUPERSID"]] = []
     # generate date if possible
    if event["DVDATEMM"] != "-8":
        event_date = date(year=int(event["DVDATEYR"]), month=int(event["DVDATEMM"]), day=1)
    else:
        event_date = None
    
    respondents_dental_visits[event["DUPERSID"]].append(
        {
            "event_id": event["EVNTIDX"],
            "date": event_date,
        }
    )

In [14]:
# Overview
total_respondents = len(fyc_lookup)
total_dental_respondents = len(respondents_dental_visits)
report_pct_of_pop(
    sub_pop=total_dental_respondents, 
    full_pop=total_respondents, 
    desc="respondents have at least 1 Dental Visit"
)

# validate event counts match
dental_match = 0
for resp, event_list in respondents_dental_visits.items():
    total_dental = len(event_list)
    if fyc_lookup[resp]["dental_care_visits_count"] == total_dental:
        dental_match += 1

report_pct_of_pop(
    sub_pop=dental_match, 
    full_pop=total_dental_respondents, 
    desc="respondents have consistent Total Dental Visits with the FYC data"
)


12,991 / 30,461 (42.65%) respondents have at least 1 Dental Visit
12,991 / 12,991 (100.00%) respondents have consistent Total Dental Visits with the FYC data


## Home Health Days

Fetch all Home Health Days. Home Health differs from other medical events, as all events are rolled up into a month. The following fields are gathered:

    - DUPERSID: UNIQUE PERSON ID
    - EVNTIDX: EVENT ID
    - HHDATEYR: EVENT DATE - YEAR
    - HHDATEMM: EVENT DATE - MONTH
    - SELFAGEN: DOES PROVIDER WORK FOR AGENCY OR SELF 
    - HHTYPE: HOME HEALTH EVENT TYPE 
    - HHDAYS: DAYS PER MONTH IN HOME HEALTH, 2018 
    - MPCELIG: MPC ELIGIBILITY FLAG 

    
First we will group Home Health days with their associated DUPERSID. Then we will verify that the aggregate counts match those from the FYC data. Then we will validate the FYC data matches the following for the following fields: 

    - home_health_provider_days_total
    - home_health_agency_days_total
    - home_health_non_agency_days_total
    - home_health_informal_days_total
    
We found that the MPCELIG flag seems to be the best method for assigning home health days to an agency, non agency or informal source. Provider days is simple the total. Using this methodology we found that the matching between the generated data and the FYC data is very close with the inconsistencies explained by the -15 cannot be computed value. 

In [15]:
HH_VARS = ["DUPERSID", "EVNTIDX", "HHDATEYR", "HHDATEMM", "SELFAGEN", "HHTYPE", "HHDAYS", "MPCELIG"]

home_health = list(
    HomeHealth18.objects.filter(DUPERSID__in=fyc_lookup.keys()).values(*HH_VARS)
)

respondents_home_health = {}
for event in home_health:
    if event["DUPERSID"] not in respondents_home_health:
        respondents_home_health[event["DUPERSID"]] = []
     
    # generate date if possible
    if event["HHDATEMM"] != "-8":
        event_date = date(year=int(event["HHDATEYR"]), month=int(event["HHDATEMM"]), day=1)
    else:
        event_date = None
    
    # classify provider type

    if event["MPCELIG"] in {"1"}:
        home_health_agency_days = int(event["HHDAYS"]) if event["HHDAYS"] != "-15" else None
    else:
        home_health_agency_days = 0

    if event["MPCELIG"] in {"2"}:
        home_health_non_agency_days = int(event["HHDAYS"]) if event["HHDAYS"] != "-15" else None
    else:
        home_health_non_agency_days = 0
        
    if event["MPCELIG"] in {"3"}:
        home_health_informal_days = int(event["HHDAYS"]) if event["HHDAYS"] != "-15" else None
    else:
        home_health_informal_days = 0
    
    respondents_home_health[event["DUPERSID"]].append(
        {
            "event_id": event["EVNTIDX"],
            "date": event_date,
            "home_health_provider_days": int(event["HHDAYS"]) if event["HHDAYS"] != "-15" else None,
            "home_health_agency_days": home_health_agency_days,
            "home_health_non_agency_days": home_health_non_agency_days,
            "home_health_informal_days": home_health_informal_days
        }
    )

In [17]:
# Overview
total_respondents = len(fyc_lookup)
total_home_health_respondents = len(respondents_home_health)
report_pct_of_pop(
    sub_pop=total_home_health_respondents, 
    full_pop=total_respondents, 
    desc="respondents have at least 1 Home Health Visit"
)

# validate total data sums match
hh_provider_days_match = 0
hh_agency_days_match = 0
hh_non_agency_days_match = 0
hh_informal_days_match = 0

for resp, event_list in respondents_home_health.items():
    # provider
    total_hh_provider_days = sum(
        [event['home_health_provider_days'] for event in event_list if event["home_health_provider_days"]]
    )
   
    if fyc_lookup[resp]["home_health_provider_days_total"] == total_hh_provider_days:
        hh_provider_days_match += 1
    
    # agency
    total_hh_agency_days = sum(
        [event['home_health_agency_days'] for event in event_list if event["home_health_agency_days"]]
    )
   
    if fyc_lookup[resp]["home_health_agency_days_total"] == total_hh_agency_days:
        hh_agency_days_match += 1
        
     # non agency
    total_hh_non_agency_days = sum(
        [event['home_health_non_agency_days'] for event in event_list if event["home_health_non_agency_days"]]
    )
   
    if fyc_lookup[resp]["home_health_non_agency_days_total"] == total_hh_non_agency_days:
        hh_non_agency_days_match += 1
    
    # informal
    total_hh_informal_days = sum(
        [event['home_health_informal_days'] for event in event_list if event["home_health_informal_days"]]
    )
   
    if fyc_lookup[resp]["home_health_informal_days_total"] == total_hh_informal_days:
        hh_informal_days_match += 1
    
report_pct_of_pop(
    sub_pop=hh_provider_days_match, 
    full_pop=total_home_health_respondents, 
    desc="respondents have consistent Home Health Provider days with the FYC data \n     (59 missing data points)"
)
report_pct_of_pop(
    sub_pop=hh_agency_days_match, 
    full_pop=total_home_health_respondents, 
    desc="respondents have consistent Agency Home Health days with the FYC data \n     (46 missing data points)"
)

report_pct_of_pop(
    sub_pop=hh_non_agency_days_match, 
    full_pop=total_home_health_respondents, 
    desc="respondents have consistent Non Agency Home Health days with the FYC data \n     (4 missing data points)"
)

report_pct_of_pop(
    sub_pop=hh_informal_days_match, 
    full_pop=total_home_health_respondents, 
    desc="respondents have consistent Informal Home Health days with the FYC data \n     (10 missing data points)"
)


1,254 / 30,461 (4.12%) respondents have at least 1 Home Health Visit
1,195 / 1,254 (95.30%) respondents have consistent Home Health Provider days with the FYC data 
     (59 missing data points)
1,208 / 1,254 (96.33%) respondents have consistent Agency Home Health days with the FYC data 
     (46 missing data points)
1,250 / 1,254 (99.68%) respondents have consistent Non Agency Home Health days with the FYC data 
     (4 missing data points)
1,244 / 1,254 (99.20%) respondents have consistent Informal Home Health days with the FYC data 
     (10 missing data points)


## Prescribed Medicines

Fetch all Prescribed Medicines. The following fields are gathered:

    - DUPERSID: UNIQUE PERSON ID
    - RXRECIDX: UNIQUE RX/PRESCRIBED MEDICINE IDENTIFIER
    
Prescribed Medicines are more complicated than event data. Here we will only verify that the total unique prescription drugs and refill match the FYC data. In the future we will need to explore the Appendix Data File which can be used to link prescription drugs with other events. This will allow us to assign a date to a prescribed drug.

In [18]:
PM_VARS = ["DUPERSID", "RXRECIDX"]

prescribed_medicine = list(
    PrescribedMedicines18.objects.filter(DUPERSID__in=fyc_lookup.keys()).values(*PM_VARS)
)

respondents_prescribed_medicine = {}
for drug in prescribed_medicine:
    if drug["DUPERSID"] not in respondents_prescribed_medicine:
        respondents_prescribed_medicine[drug["DUPERSID"]] = 0
    respondents_prescribed_medicine[drug["DUPERSID"]] += 1

In [19]:
# Overview
total_respondents = len(fyc_lookup)
total_prescribed_medicinel_respondents = len(respondents_prescribed_medicine)
report_pct_of_pop(
    sub_pop=total_prescribed_medicinel_respondents, 
    full_pop=total_respondents, 
    desc="respondents have at least 1 Prescribed Medicine"
)

# validate event counts match
prescribed_medicine_match = 0
for resp, count in respondents_prescribed_medicine.items():
    if fyc_lookup[resp]["prescription_drugs_total"] == count:
        prescribed_medicine_match += 1

report_pct_of_pop(
    sub_pop=prescribed_medicine_match, 
    full_pop=total_prescribed_medicinel_respondents, 
    desc="respondents have consistent Prescription Medicine Totals with the FYC data"
)

18,145 / 30,461 (59.57%) respondents have at least 1 Prescribed Medicine
18,145 / 18,145 (100.00%) respondents have consistent Prescription Medicine Totals with the FYC data
